In [56]:
from IPython.core.magic import register_cell_magic

@register_cell_magic
def write_and_run(line, cell):
    argz = line.split()
    file = argz[-1]
    mode = 'w'
    if len(argz) == 2 and argz[0] == '-a':
        mode = 'a'
    with open(file, mode) as f:
        f.write(cell)
    get_ipython().run_cell(cell)

# **Import libraries and modules**

## Import libraries

In [57]:
import os
import sys
import pathlib
import shutil
import math

import tensorflow as tf
from tensorflow import keras
from keras import backend as K
from keras.layers import *
from keras.losses import SparseCategoricalCrossentropy
from keras.regularizers import l2
from tensorflow.keras.optimizers import SGD, Adam

from sklearn.metrics import classification_report, ConfusionMatrixDisplay

import seaborn as sn

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [58]:
def scheduler(epochs, lr):
    S = [200, 250, 300, 350, 400]
    print([c == epochs for c in S])
    if [c == epochs for c in S].count(True) > 0:
        return lr * 0.1
    return lr

scheduler(200, 0.1)

[True, False, False, False, False]


0.010000000000000002

## Import modules

In [59]:
sys.path.insert(0, '../')

from generate_version import generate_version

# **Prepare the dataset for loading**

## Helper functions

In [60]:
def create_label_dir(df, dir='../gdsc-ai-challenge/train'):
    """Use Dataframe contains labels for each image and path to the directory

    contains the unlabeled dataset to rebuild directory into labeled subdirectories.

    Returns all the label and number of classes in the dataset.

    Keyword arguments:

    df -- The Dataframe contains images' names and labels.

    dir -- Path to the main directory (default to ../gdsc-ai-challenge/train)
    """
    class_names = np.sort(df['label'].unique())
    number_of_classes = len(class_names)

    if not os.path.exists(dir):
        return class_names, number_of_classes

    for class_name in class_names:
        subdir = pathlib.Path(os.path.join(dir, class_name))
        if subdir.exists():
            continue
        else:
            subdir.mkdir()
    
    return class_names, number_of_classes

def sort_data(df, dir='../gdsc-ai-challenge/train'):
    """Use Dataframe to move each unlabeled image to the correct label's subdirectory.

    df -- The Dataframe contains images' names and labels.

    dir -- Path to the main directory (default to ../gdsc-ai-challenge/train) 
    """
    if not os.path.exists(dir):
        return
    
    unlabeled_dir = os.path.join(dir, 'train')

    for image_dir in [str(img) for img in list(pathlib.Path(unlabeled_dir).glob('*.png'))]:
        id = int(image_dir.removeprefix(unlabeled_dir).removesuffix('.png'))
        label = df['label'][id - 1]
        dest_path = os.path.join(dir, label, str(id) + '.png')
        shutil.move(image_dir, dest_path)

## Prepare the dataset

In [61]:
label_df = pd.read_csv('../gdsc-ai-challenge/trainLabels.csv')

class_names, number_of_classes = create_label_dir(label_df)
sort_data(label_df)

# **Data preprocessing and augmentation**

## Helper functions

In [63]:
def split_dataset(ds, ds_size, train_split=0.8, val_split=0.1, test_split=0.1):
    """Split the dataset into three subsets: train, validation (dev) and test set.

    Returns three tuples, containing each subset with its size.

    Keyword arguments:

    ds -- tf.data.Dataset object

    ds_size -- size of the dataset

    train_split -- percentage to split into train set (default to 0.8)

    val_split -- percentage to split into validation set (default to 0.1)

    test_split -- percentage to split into test set (default to 0.1)
    """
    assert (train_split + test_split + val_split) == 1
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return (train_ds, train_size), (val_ds, val_size), (test_ds, ds_size - val_size - train_size)

def configure(ds, ds_size, batch_size=32, shuffle=False, augment=False):
    """Configure the given dataset for better performance (by caching, prefetching and then batching the dataset)

    and perform preprocessing to the images in the given dataset.

    Returns the optimized dataset.

    Keyword arguments:

    ds -- tf.data.Dataset object

    ds_size -- size of the dataset

    batch_size -- size of each batch (default to 32)

    shuffle -- whether to shuffle the dataset (default to False)

    augment -- whether to perform data augmentation to the dataset (default to False)
    """

    AUTOTUNE = tf.data.AUTOTUNE
    rescale = keras.layers.Rescaling(1.0/255)
    data_augmentation = keras.Sequential([
        keras.layers.RandomFlip('horizontal'),
        keras.layers.RandomRotation(0.05, fill_mode='nearest')
    ])

    ds = ds.map(lambda x, y: (rescale(x), y),
                num_parallel_calls=AUTOTUNE)

    ds = ds.cache()
    if shuffle:
        ds = ds.shuffle(buffer_size=int(ds_size * 0.6))
    
    ds = ds.batch(batch_size)

    if augment:
        with tf.device('/cpu:0'):
            #only perform data augmentation on train set
            ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y),
                                    num_parallel_calls=AUTOTUNE)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds

## Load the dataset using *image_dataset_from_directory()*

In [64]:
ds = tf.keras.utils.image_dataset_from_directory(
    '../gdsc-ai-challenge/train',
    color_mode='rgb',
    batch_size=None,
    image_size=(32,32),
    seed=42
)

ds_size = ds.cardinality().numpy()

Found 50000 files belonging to 10 classes.


## Split the dataset and Preprocess the dataset

In [66]:
(train_ds, train_size), (val_ds, val_size), (test_ds, test_size) = split_dataset(ds, ds_size, train_split=0.7, val_split=0.2, test_split=0.1)

train_ds = configure(train_ds, train_size, augment=True, shuffle=True)
val_ds = configure(val_ds, val_size)
test_ds = configure(test_ds, test_size)

# **Create version-controlled folder for weights file**

In [67]:
version = input("""Create new folder?

                (Y/n)    
                """)

new_version, path, weights_save_path = generate_version('../Model/aiseries', 'weights.best.hdf5', version.lower() == 'y')
_, _, report_save_path = generate_version('../TrainingReport', new_version=version.lower() == 'y')

# **Build a model**

In [68]:
%%write_and_run {path}/model.py
import tensorflow as tf
from tensorflow import keras
from keras import regularizers
import os

def create_model(path_to_weights='', load_weights=True):
    """Function to create a model

    Returns a compiled and optionally loaded model

    Keyword arguments:

    path_to_weights -- (Optional, only used when load_weights is True) -- Path to weight file (.hdf5 files)

    load_weights -- Whether to load weights or not (default to True)
    """
    if (load_weights):
        assert(path_to_weights is not None and 
           os.path.isfile(path_to_weights)), "path_to_weights must exist and not be empty if load_weights is True, otherwise change load_weights to False"

    model = keras.models.Sequential([
        Input((32,32,3)),
        Dropout(0.2),
        Conv2D(96, (3,3), padding='same',
                            kernel_regularizer=l2(1e-3),
#                             activity_regularizer=l2(1e-3),
                            kernel_initializer='he_normal',
                            activation='relu'),
        BatchNormalization(),
        Conv2D(96, (3,3), padding='same',
                            kernel_regularizer=l2(1e-3),
#                             activity_regularizer=l2(1e-3),
                            kernel_initializer='he_normal',
                            activation='relu'),
        BatchNormalization(),
        Conv2D(96, (3,3), padding='same', strides=(2,2),
                            kernel_regularizer=l2(1e-3),
#                             activity_regularizer=l2(1e-3),
                            kernel_initializer='he_normal',
                            activation='relu'),
        BatchNormalization(),
        Dropout(0.5),

        Conv2D(192, (3,3), padding='same',
                            kernel_regularizer=l2(1e-3),
#                             activity_regularizer=l2(1e-3),
                            kernel_initializer='he_normal',
                            activation='relu'),
        BatchNormalization(),
        Conv2D(192, (3,3), padding='same',
                            kernel_regularizer=l2(1e-3),
#                             activity_regularizer=l2(1e-3),
                            kernel_initializer='he_normal',
                            activation='relu'),
        BatchNormalization(),
        Conv2D(192, (3,3), padding='same', strides=(2,2),
                            kernel_regularizer=l2(1e-3),
#                             activity_regularizer=l2(1e-3),
                            kernel_initializer='he_normal',
                            activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        
        Conv2D(192, (3,3), padding='same',
                            kernel_regularizer=l2(1e-3),
#                             activity_regularizer=l2(1e-3),
                            kernel_initializer='he_normal',
                            activation='relu'),
        BatchNormalization(),
        Conv2D(192, (1,1), padding='same',
                            kernel_regularizer=l2(1e-3),
#                             activity_regularizer=l2(1e-3),
                            kernel_initializer='he_normal',
                            activation='relu'),
        BatchNormalization(),
        Conv2D(10, (1,1), padding='same',
                            kernel_regularizer=l2(1e-3),
#                             activity_regularizer=l2(1e-3),
                            kernel_initializer='he_normal',
                            activation='relu'),
        BatchNormalization(),
        GlobalAveragePooling2D(),
        
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])

    if load_weights:
        model.load_weights(path_to_weights)

    model.compile(optimizer=Adam(learning_rate=0.001),
                                 loss='sparse_categorical_crossentropy',
                                 metrics=[
                                 SparseCategoricalCrossentropy(name='sparse'),
                                 'accuracy'])

    return model

In [70]:
model = create_model(load_weights=False)

tf.keras.utils.plot_model(model, os.path.join(path, 'model.png'), show_shapes=True)

model.summary()

[array([[[[-1.2327514e-01,  1.8758246e-01,  1.3366115e-01, ...,
          -3.9569274e-01, -2.2093794e-01,  2.2485802e-01],
         [ 8.5749123e-03,  8.2057893e-02, -3.0778140e-01, ...,
           2.9060813e-02,  4.0433702e-01,  9.1361411e-02],
         [ 6.4341351e-02, -2.1280487e-01,  4.2741010e-01, ...,
           8.1815846e-02, -2.0387091e-03, -2.5813416e-01]],

        [[ 2.8825894e-01,  1.8547261e-01,  3.8966179e-01, ...,
           4.5996919e-01,  5.2945131e-01,  4.8165825e-01],
         [ 2.3836413e-01,  5.3261894e-01,  8.8013045e-02, ...,
           3.4440061e-04, -6.0009710e-02, -4.9282849e-01],
         [ 7.6590963e-02, -3.0192828e-01, -2.6525411e-01, ...,
           2.7328792e-01, -4.4015843e-02,  2.2303690e-01]],

        [[-2.0968381e-01, -7.0211850e-02, -1.8465038e-01, ...,
          -3.0439100e-01,  1.2702802e-01, -4.3784800e-01],
         [ 3.0657706e-01,  4.9449268e-01, -1.4165045e-01, ...,
          -1.9376539e-01, -1.3100624e-01, -1.0987287e-01],
         [-4.973335

# **Training session**

## Design callbacks to stop training

In [ ]:
#callback to save weights with the minimum loss value
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=weights_save_path,
                                                               monitor='val_loss',
                                                               mode='min',
                                                               save_best_only=True)

class EarlyStoppingOnMaxAccuracy(keras.callbacks.Callback):
    def __init__(self, patience=0):
        super(EarlyStoppingOnMaxAccuracy, self).__init__()
        self.patience = patience
        self.best_weights = None
    
    def on_train_begin(self, logs=None):
        self.epochs_waited = 0
        self.stopped_epoch = 0
        self.best = - np.Inf
    
    def on_epoch_end(self, epoch, logs=None):
        print(self.model.optimizer.grad)
        current_val_accuracy = logs.get("val_accuracy")
        current_accuracy = logs.get("accuracy")

        if math.isclose(current_val_accuracy, current_accuracy, rel_tol=0.05):
            self.model.stop_training = True
        elif current_val_accuracy > 0.6 and np.greater(current_val_accuracy, self.best):
            self.best = current_val_accuracy
            self.epochs_waited = 0
            self.best_weights = self.model.get_weights()
        else:
            self.epochs_waited += 1
            if self.epochs_waited >= self.patience:
                self.stopped_epoch = epoch
                self.stop_training = True
                print("Restoring model weights from the end of the best epoch.")
                self.model.set_weights(self.best_weights)
    
    def on_train_end(self, logs=None):
        if self.stopped_epoch > 0:
            print("Epoch %05d: early stopping" % (self.stopped_epoch + 1))

def recompileWithSGD(model, lr=0.001, momentum=0.9, loss_fn='sparse_categorical_crossentropy', 
                     train_data=None, epochs=None, callbacks=[], validation_data=None):
    model.compile(optimizer=SGD(learning_rate=lr, momentum=momentum), 
                  loss=loss_fn, 
                  metrics=[
                      SparseCategoricalCrossentropy(name='sparse'),
                      'accuracy'
                  ])

    model.fit(train_data, epochs=epochs, callbacks=callbacks, validation_data=validation_data)
    return model

class AdamToSGD(keras.callbacks.EarlyStopping):
    def __init__(self):
        self.restore_best_weights = True
        self.phase = 'Adam'
        self.best_weights = None

    def on_train_begin(self, logs=None):
        self.weights = None
        self.bias_corrected_exponential_average = None
        self.lambda_k = 0
        self.lr = self.model.optimizer.lr

    def on_epoch_end(self, epoch, logs={}):
        lr = self.model.optimizer.lr
        if self.weights is not None:
            p_k = self.model.get_weights() - self.weights
            g_k
        else:
            self.weights = self.model.get_weights()
        self.bias_corrected_exponential_average = lr / (1. - self.model.optimizer.beta_2)
        if (K.abs(self.bias_corrected_exponential_average - lr) < model.optimizer.epsilon) is not None:
            if self.restore_best_weights:
                 self.best_weights = self.model.get_weights()
        else:
            self.stopped_epoch = epoch
            self.model.stop_training = True
            print('Requirement met, changing to Stoichastic Gradient Descent')
            if self.restore_best_weights:
                if self.verbose > 0:
                    print('Restoring model weights from the end of the best epoch')
                    self.model.set_weights(self.best_weights)
    
    def on_train_end(self, logs=None):
        recompileWithSGD(self.model, self.bias_corrected_exponential_average, self.model.optimizer.beta_1,
                        train_data=train_ds, epochs=200, 
                        callbacks=[EarlyStoppingOnMaxAccuracy(), model_checkpoint_callback],
                        validation_data=val_ds
                        )
        

In [ ]:
history = model.fit(train_ds, 
                    epochs=350, 
                    callbacks=[EarlyStoppingOnMaxAccuracy(), AdamToSGD(), model_checkpoint_callback], 
                    validation_data=val_ds)

1094/1094 [==============================] - ETA: 0s - loss: 1.8884 - sparse_categorical_crossentropy: 1.6350 - accuracy: 0.3907

AttributeError: 'Adam' object has no attribute 'grad'

# **Model evaluation**

## Evaluate on the test set using the best model

In [ ]:
model.load_weights(weights_save_path)

model.evaluate(test_ds)

## Evaluate based on training's metrics history

### 1. Based on loss value

In [ ]:
number_of_epochs = len(history.history['loss'])

In [ ]:
plt.plot(history.history['loss'], color='red', label='Train loss')
plt.plot(history.history['val_loss'], color='blue', label='Validation loss')

plt.xticks(np.arange(number_of_epochs, step=4))
plt.legend()
plt.show()

### 2. Based on accuracy

In [ ]:
plt.plot(history.history['accuracy'], color='red', label='Training accuracy')
plt.plot(history.history['val_accuracy'], color='blue', label='Validation accuracy')

plt.xticks(np.arange(number_of_epochs, step=4))
plt.legend()
plt.show()

## Evaluate with Confusion Matrix and Classification Report

### Generate actual and predicted value

In [ ]:
y_true = np.concatenate([y for _, y in test_ds], axis=0)

Y_pred = model.predict(test_ds)
y_pred = np.argmax(Y_pred, axis=1)

### Plot Confusion matrix

In [ ]:
ConfusionMatrixDisplay.from_predictions(y_true, y_pred,
                                        display_labels=[class_name.capitalize() for class_name in class_names],
                                        cmap='Blues')
plt.xticks(rotation=60)
plt.savefig(os.path.join(report_save_path, 'confusion_matrix.pdf'))
plt.show()

### Plot classification report

In [ ]:
clf_rep = classification_report(y_true, y_pred, 
                                target_names=[class_name.capitalize() for class_name in class_names], 
                                output_dict=True)

sn.heatmap(pd.DataFrame(clf_rep).iloc[:-1,:].T, annot=True)
plt.savefig(os.path.join(report_save_path, 'report.pdf'))